<a href="https://colab.research.google.com/github/pabloac31/TFG/blob/master/cifar10_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/pabloac31/TFG.git
%cd TFG

Cloning into 'TFG'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 110 (delta 42), reused 28 (delta 12), pack-reused 41
Receiving objects: 100% (110/110), 79.06 MiB | 28.77 MiB/s, done.
Resolving deltas: 100% (48/48), done.
/content/TFG


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [11]:
! git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/pabloac31/TFG
   9c9effe..62a2daa  master     -> origin/master
Updating 9c9effe..62a2daa
Fast-forward
 adversarial_attacks.py | 38 ++++++++++++++++----------------------
 1 file changed, 16 insertions(+), 22 deletions(-)


In [0]:
from cifar10_models import *
from utils import *
from adversarial_attacks import *

In [0]:
iv3 = inception_v3(pretrained=True)

In [5]:
# Define what device we are using
use_cuda=True
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  True


In [6]:
test_loader = testloader_cifar10('./data', 256)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [0]:
test_model(iv3, device, test_loader)

100%|██████████| 40/40 [00:21<00:00,  1.83it/s]


0.9541

In [6]:
adv_loader = testloader_cifar10('./data', 1)

Extracting ./data/cifar-10-python.tar.gz to ./data


In [11]:
test_fgsm(iv3, device, './data/img1.png', 0.56)

Original image prediction:  1


TypeError: ignored

In [16]:
params = {'epsilon': 0.4}
adv_examples = attack_model(iv3, device, adv_loader, 'fgsm', params, iters=10)




  0%|          | 0/10000 [00:00<?, ?it/s]


  0%|          | 1/10000 [00:00<25:04,  6.65it/s]

>>>>>> Using FGSM <<<<<<





  0%|          | 2/10000 [00:00<24:37,  6.77it/s]


  0%|          | 4/10000 [00:00<21:28,  7.76it/s]


  0%|          | 5/10000 [00:00<21:15,  7.84it/s]


  0%|          | 6/10000 [00:00<20:48,  8.00it/s]


  0%|          | 7/10000 [00:00<20:13,  8.24it/s]


  0%|          | 8/10000 [00:00<20:20,  8.19it/s]


  0%|          | 9/10000 [00:01<19:43,  8.44it/s]


  0%|          | 10/10000 [00:01<19:23,  8.59it/s]


======== RESULTS ========
Test Accuracy = 5 / 10 = 0.5
Average confidence = 0.7559603750705719
Average time = 0.049154353141784665
Average magnitude of perturbations = 19.64453887939453
Model robustness = 0.3451698571443558
